In [35]:
import pandas as pd
import numpy as np
import helpers
import geopy.distance

In [36]:
helpers.download_bike_data()
helpers.download_covid_data()
helpers.download_weather_data()

All divvy bike files downloaded successfully
Covid data downloaded succesfully
Weather data downloaded succesfully


In [37]:
bikes = helpers.assemble_bike_data()
covid = pd.read_csv('data/covid/covid_data.csv', index_col=0)
weather = pd.read_csv('data/weather/weather_data.csv', index_col=0)

### Bike Data Exploration

In [38]:
bikes['started_at'] = pd.to_datetime(bikes['started_at'])
bikes['ended_at'] = pd.to_datetime(bikes['ended_at'])

In [39]:
bikes.head(1)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86,Lincoln Ave & Diversey Pkwy,152.0,41.8964,-87.661,41.9322,-87.6586,member


In [40]:
#Percentage of rides without returns
bikes['end_station_name'].isna().sum() / bikes.shape[0]


0.10841628759141815

In [41]:
#Percentage of members vs casual users
bikes['member_casual'].value_counts(normalize=True)

member    0.571036
casual    0.428964
Name: member_casual, dtype: float64

In [42]:
#Types of bicycles used
bikes['rideable_type'].value_counts(normalize=True)

classic_bike     0.397413
electric_bike    0.318716
docked_bike      0.283871
Name: rideable_type, dtype: float64

In [43]:
#Average number of rides per day
pd.to_datetime(bikes['started_at']).dt.date.value_counts().mean()

12952.328263624842

In [44]:
# Ride Duration
bikes['minutes'] = (bikes['ended_at'] - bikes['started_at']).dt.total_seconds() / 60

In [45]:
# Time of day
bikes['time_of_day'] = None
bikes.loc[((bikes['started_at'].dt.hour >= 5) & (bikes['started_at'].dt.hour <= 10)), 'time_of_day'] = 'morning'
bikes.loc[((bikes['started_at'].dt.hour >= 11) & (bikes['started_at'].dt.hour <= 16)), 'time_of_day'] = 'mid_day'
bikes.loc[((bikes['started_at'].dt.hour >= 17) & (bikes['started_at'].dt.hour <= 22)), 'time_of_day'] = 'evening'
bikes.loc[((bikes['started_at'].dt.hour >= 23) | (bikes['started_at'].dt.hour <= 4)), 'time_of_day'] = 'night'
bikes['time_of_day'].value_counts(normalize=True)

mid_day    0.403784
evening    0.358147
morning    0.184135
night      0.053934
Name: time_of_day, dtype: float64

In [46]:
# Distance traveled
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.
    FROM: https://stackoverflow.com/questions/43577086/
    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

bikes['km_between_stations'] = haversine(bikes['start_lat'],
                                                             bikes['start_lng'], 
                                                             bikes['end_lat'],
                                                             bikes['end_lng'])

In [47]:
# Percent of bikes returned to same station
len(bikes[bikes['km_between_stations'] == 0]) / len(bikes)

0.07355666244951874

In [53]:
bikes['date'] = pd.to_datetime(bikes['started_at']).dt.date

### Weather

In [54]:
weather.reset_index(inplace=True)
weather = weather.rename(columns={'time':'date'})
weather['date'] = pd.to_datetime(weather['date'])
weather.head(1)

,index,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,0,2020-04-01,6.2,2.2,11.7,0.0,0.0,25.0,12.5,NaN,1019.2,NaN


In [55]:
weather.describe()

,index,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,728.000000,730.000000,0.0,728.000000,0.0
mean,364.500000,11.927534,7.367534,16.532466,2.822192,18.915068,186.729396,15.881781,NaN,1016.767445,NaN
std,210.877136,10.956767,10.683166,11.610957,8.093148,73.366423,100.749323,5.486643,NaN,6.945882,NaN
min,0.000000,-17.500000,-21.100000,-14.400000,0.000000,0.000000,0.000000,3.800000,NaN,995.200000,NaN
25%,182.250000,3.125000,-0.600000,7.200000,0.000000,0.000000,84.750000,12.100000,NaN,1012.200000,NaN
50%,364.500000,11.950000,6.700000,17.200000,0.000000,0.000000,207.500000,15.250000,NaN,1016.150000,NaN
75%,546.750000,22.100000,17.800000,27.200000,1.300000,0.000000,263.500000,18.900000,NaN,1021.200000,NaN
max,729.000000,29.700000,25.000000,35.600000,100.300000,660.000000,360.000000,37.900000,NaN,1037.600000,NaN


### Covid

In [56]:
covid = covid.rename(columns={'Date':'date'})
covid['date'] = pd.to_datetime(covid['date'])
covid.head(1)

,date,Cases - Total,Deaths - Total,Hospitalizations - Total,Cases - Age 0-17,Cases - Age 18-29,Cases - Age 30-39,Cases - Age 40-49,Cases - Age 50-59,Cases - Age 60-69,...,Hospitalizations - Age Unknown,Hospitalizations - Female,Hospitalizations - Male,Hospitalizations - Unknown Gender,Hospitalizations - Latinx,Hospitalizations - Asian Non-Latinx,Hospitalizations - Black Non-Latinx,Hospitalizations - White Non-Latinx,Hospitalizations - Other Race Non-Latinx,Hospitalizations - Unknown Race/Ethnicity
0,2021-12-26,4459,11,208.0,628,1204,1017,702,457,263,...,0.0,120.0,88.0,0.0,34.0,5.0,116.0,42.0,8.0,3.0


In [52]:
covid.describe()

,Cases - Total,Deaths - Total,Hospitalizations - Total,Cases - Age 0-17,Cases - Age 18-29,Cases - Age 30-39,Cases - Age 40-49,Cases - Age 50-59,Cases - Age 60-69,Cases - Age 70-79,...,Hospitalizations - Age Unknown,Hospitalizations - Female,Hospitalizations - Male,Hospitalizations - Unknown Gender,Hospitalizations - Latinx,Hospitalizations - Asian Non-Latinx,Hospitalizations - Black Non-Latinx,Hospitalizations - White Non-Latinx,Hospitalizations - Other Race Non-Latinx,Hospitalizations - Unknown Race/Ethnicity
count,873.000000,873.000000,868.000000,873.000000,873.000000,873.000000,873.000000,873.000000,873.000000,873.000000,...,868.000000,868.000000,868.000000,868.000000,868.000000,868.000000,868.000000,868.000000,868.000000,868.000000
mean,750.651775,8.890034,50.905530,121.782360,175.175258,150.951890,109.280641,88.002291,58.617411,29.257732,...,0.010369,25.387097,25.511521,0.006912,13.316820,1.481567,23.223502,10.408986,1.657834,0.816820
std,1116.865139,10.602354,53.323502,230.423166,267.719764,225.463101,156.878877,125.005147,82.391047,38.913439,...,0.101356,26.064819,27.871903,0.082901,15.717733,2.499067,25.410735,10.680200,2.327157,1.230089
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,256.000000,2.000000,21.000000,30.000000,59.000000,52.000000,35.000000,28.000000,18.000000,9.000000,...,0.000000,10.000000,9.000000,0.000000,4.000000,0.000000,8.000000,4.000000,0.000000,0.000000
50%,436.000000,5.000000,31.000000,64.000000,105.000000,89.000000,64.000000,49.000000,32.000000,16.000000,...,0.000000,16.000000,15.000000,0.000000,8.000000,1.000000,14.500000,7.000000,1.000000,0.000000
75%,816.000000,12.000000,62.000000,118.000000,186.000000,164.000000,124.000000,103.000000,72.000000,37.000000,...,0.000000,31.000000,31.000000,0.000000,16.000000,2.000000,29.000000,13.000000,2.000000,1.000000
max,10433.000000,58.000000,698.000000,2821.000000,2303.000000,2009.000000,1393.000000,1110.000000,743.000000,336.000000,...,1.000000,340.000000,358.000000,1.000000,219.000000,51.000000,236.000000,151.000000,29.000000,12.000000


### Compact Table

In [156]:
# compact_table = pd.DataFrame(pd.date_range(helpers.START_DATE,helpers.END_DATE, freq='D'),columns=['date'])

In [228]:
selection_list = []
# Group and count categorical features.
for time in bikes['time_of_day'].unique():
    for member in bikes['member_casual'].unique():
        for ride_type in bikes['rideable_type'].unique():
            iter_filter = (bikes['member_casual'] == member) & (bikes['time_of_day'] == time) & (bikes['rideable_type'] == ride_type)
            selection = bikes[iter_filter]
            count = selection.groupby('date').size().reset_index().rename(columns={0:'rides'})
            means = selection[['date', 'km_between_stations', 'minutes']].groupby('date').mean().reset_index()
            means = means.rename(columns={'km_between_stations':'avg_km_between_stations', 'minutes':'avg_minutes'})
            selection = pd.merge(count, means, on='date')
            selection['time_of_day'] = time  
            selection['membership'] = member
            selection['ride_type'] = ride_type
            selection_list.append(selection)

In [233]:
compact_table = pd.concat(selection_list)
compact_table.fillna(0, inplace=True)
compact_table.to_csv('compact_bike_data.csv')

In [237]:
covid.rename(columns={'Cases - Total':'cases', 'Deaths - Total':'deaths', 'Hospitalizations - Total':'hospitalizations'}, inplace=True)
covid[['date','cases', 'deaths', 'hospitalizations']].to_csv('covid_data.csv')

In [238]:
weather[['date','tavg', 'prcp', 'snow', 'wspd']].to_csv('weather_data.csv')

In [175]:
compact_table.fillna(0, inplace=True)
compact_table.to_csv('compact_bike_data.csv')